# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV
import warnings

warnings.filterwarnings('ignore')

## Подготовка данных

In [2]:
data = pd.read_csv('/datasets/autos.csv')
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

Проверим данные на дубликаты и удалим их.

In [4]:
data.duplicated().sum()

4

In [5]:
data = data.drop_duplicates()
data.duplicated().sum()

0

Удалим неинформативные стобцы(которые не потребуются для модели)

In [6]:
data = data.drop(['DateCrawled', 'RegistrationMonth', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'], axis=1)
data.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,Repaired
0,480,NaN,1993,manual,0,golf,150000,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no


Обработаем пропуски

In [7]:
data.isna().sum()

Price                   0
VehicleType         37490
RegistrationYear        0
Gearbox             19833
Power                   0
Model               19705
Kilometer               0
FuelType            32895
Brand                   0
Repaired            71154
dtype: int64

Поставим заглушки на места пропусков в этих столбцах.

In [8]:
data['VehicleType'] = data['VehicleType'].fillna('unknown')
data['Gearbox'] = data['Gearbox'].fillna('unknown')
data['Model'] = data['Model'].fillna('unknown')
data['FuelType'] = data['FuelType'].fillna('unknown')
data['Repaired'] = data['Repaired'].fillna('unknown')

In [9]:
data.isna().sum()

Price               0
VehicleType         0
RegistrationYear    0
Gearbox             0
Power               0
Model               0
Kilometer           0
FuelType            0
Brand               0
Repaired            0
dtype: int64

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354365 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354365 non-null  int64 
 1   VehicleType       354365 non-null  object
 2   RegistrationYear  354365 non-null  int64 
 3   Gearbox           354365 non-null  object
 4   Power             354365 non-null  int64 
 5   Model             354365 non-null  object
 6   Kilometer         354365 non-null  int64 
 7   FuelType          354365 non-null  object
 8   Brand             354365 non-null  object
 9   Repaired          354365 non-null  object
dtypes: int64(4), object(6)
memory usage: 29.7+ MB


Проверим аномальные значения.

In [11]:
data.describe()

,Price,RegistrationYear,Power,Kilometer
count,354365.000000,354365.000000,354365.000000,354365.000000
mean,4416.679830,2004.234481,110.093816,128211.363989
std,4514.176349,90.228466,189.851330,37905.083858
min,0.000000,1000.000000,0.000000,5000.000000
25%,1050.000000,1999.000000,69.000000,125000.000000
50%,2700.000000,2003.000000,105.000000,150000.000000
75%,6400.000000,2008.000000,143.000000,150000.000000
max,20000.000000,9999.000000,20000.000000,150000.000000


Можем заметить аномальные значения в стобцах RegistrationYear и Power. Обработаем их.

In [12]:
data = data.query('RegistrationYear > 1900 and RegistrationYear < 2017')

In [13]:
data = data.query('Power < 2000')

Нулевую мощность и нулевую цену заменим на медианное значение этих стобоцов, потому что удаление строк привело бы к большой потери данных.

In [14]:
data['Power'] = data['Power'].replace(0, data['Power'].median())
data['Price'] = data['Price'].replace(0, data['Price'].median())

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 339657 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Price             339657 non-null  float64
 1   VehicleType       339657 non-null  object 
 2   RegistrationYear  339657 non-null  int64  
 3   Gearbox           339657 non-null  object 
 4   Power             339657 non-null  float64
 5   Model             339657 non-null  object 
 6   Kilometer         339657 non-null  int64  
 7   FuelType          339657 non-null  object 
 8   Brand             339657 non-null  object 
 9   Repaired          339657 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 28.5+ MB


In [16]:
data.describe()

,Price,RegistrationYear,Power,Kilometer
count,339657.000000,339657.000000,339657.000000,339657.000000
mean,4554.084579,2002.482519,119.615927,128088.277292
std,4489.445806,7.090961,60.238615,37891.546696
min,1.000000,1910.000000,1.000000,5000.000000
25%,1240.000000,1999.000000,84.000000,125000.000000
50%,2799.000000,2003.000000,105.000000,150000.000000
75%,6500.000000,2007.000000,143.000000,150000.000000
max,20000.000000,2016.000000,1999.000000,150000.000000


В столбце FuelType заменим gasoline на petrol т.к. они обозначают бензин.

In [17]:
data['FuelType'].value_counts()

petrol      209939
gasoline     96283
unknown      27227
lpg           5158
cng            541
hybrid         225
other          195
electric        89
Name: FuelType, dtype: int64

In [18]:
data['FuelType'] = data['FuelType'].replace(['gasoline'], 'petrol')

На этапе предобработки мы избавились от ненужных столбцов, удалили дубликаты, заполнили пропущенные значения и избавились от аномалий.

## Обучение моделей

С помощью get_dummeis приведем категориальные значения стобцов.

In [19]:
#data_ohe = pd.get_dummies(data, drop_first=True)

In [20]:
features_oe = data.drop('Price', axis=1)
target_oe = data['Price']

In [21]:
enc = OrdinalEncoder()
enc.fit(features_oe)
features_oe = pd.DataFrame(enc.transform(features_oe))

In [22]:
#features_ohe = data_ohe.drop('Price', axis=1)
#target_ohe = data_ohe['Price']

In [23]:
features_train_oe, features_test_oe, target_train_oe, target_test_oe = train_test_split(features_oe, target_oe,
                                                                           test_size=0.25, random_state=12345)

In [24]:
features_train_oe.shape

(254742, 9)

In [25]:
len(target_train_oe)

254742

In [26]:
features_test_oe.shape

(84915, 9)

In [27]:
len(target_test_oe)

84915

In [29]:
features = data.drop('Price', axis=1)
target = data['Price']

Для модели LightGBM нам потребуется изменить тип категориальных переменных

In [30]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 339657 entries, 0 to 354368
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   VehicleType       339657 non-null  object 
 1   RegistrationYear  339657 non-null  int64  
 2   Gearbox           339657 non-null  object 
 3   Power             339657 non-null  float64
 4   Model             339657 non-null  object 
 5   Kilometer         339657 non-null  int64  
 6   FuelType          339657 non-null  object 
 7   Brand             339657 non-null  object 
 8   Repaired          339657 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 25.9+ MB


In [31]:
categorical = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']
features[categorical] = features[categorical].astype('category')

In [32]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 339657 entries, 0 to 354368
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   VehicleType       339657 non-null  category
 1   RegistrationYear  339657 non-null  int64   
 2   Gearbox           339657 non-null  category
 3   Power             339657 non-null  float64 
 4   Model             339657 non-null  category
 5   Kilometer         339657 non-null  int64   
 6   FuelType          339657 non-null  category
 7   Brand             339657 non-null  category
 8   Repaired          339657 non-null  category
dtypes: category(6), float64(1), int64(2)
memory usage: 12.6 MB


In [33]:
features_train, features_test, target_train, target_test = train_test_split(features, target,
                                                                           test_size=0.25, random_state=12345)

## Анализ моделей

Начнем с обучения моделей LightGBM. Для этого воспользуемся GridSearchCV.

In [34]:
model_lgb = lgb.LGBMRegressor(random_state=12345)
params = {'num_leaves': [25, 50], 'n_estimators': [50]}
m = GridSearchCV(estimator=model_lgb, param_grid=params, scoring='neg_root_mean_squared_error', cv=2, verbose=3)
m.fit(features_train, target_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV 1/2] END .................n_estimators=50, num_leaves=25; total time= 2.7min
[CV 2/2] END .................n_estimators=50, num_leaves=25; total time= 2.7min
[CV 1/2] END .................n_estimators=50, num_leaves=50; total time= 5.2min
[CV 2/2] END .................n_estimators=50, num_leaves=50; total time= 2.3min


GridSearchCV(cv=2, estimator=LGBMRegressor(random_state=12345),
             param_grid={'n_estimators': [50], 'num_leaves': [25, 50]},
             scoring='neg_root_mean_squared_error', verbose=3)

In [35]:
m.best_estimator_

LGBMRegressor(n_estimators=50, num_leaves=50, random_state=12345)

In [36]:
m.best_score_

-1736.2909283276144

Получили оценку RMSE для LightGMB 1736.2909283276144. Что удовлетворяют условию нашей задачи. Оценка достигается при n_estimators=50 и num_leaves=50

Случайный лес.

In [37]:
model_forest = RandomForestRegressor(random_state=12345)
params2 = {'max_depth': [5,10], 'n_estimators': [50, 100]}
m2 = GridSearchCV(estimator=model_forest, param_grid=params2, scoring='neg_root_mean_squared_error', cv=2, verbose=3)
m2.fit(features_train_oe, target_train_oe)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV 1/2] END ...................max_depth=5, n_estimators=50; total time=   4.7s
[CV 2/2] END ...................max_depth=5, n_estimators=50; total time=   4.7s
[CV 1/2] END ..................max_depth=5, n_estimators=100; total time=   9.4s
[CV 2/2] END ..................max_depth=5, n_estimators=100; total time=   9.5s
[CV 1/2] END ..................max_depth=10, n_estimators=50; total time=   8.4s
[CV 2/2] END ..................max_depth=10, n_estimators=50; total time=   8.4s
[CV 1/2] END .................max_depth=10, n_estimators=100; total time=  16.6s
[CV 2/2] END .................max_depth=10, n_estimators=100; total time=  16.7s


GridSearchCV(cv=2, estimator=RandomForestRegressor(random_state=12345),
             param_grid={'max_depth': [5, 10], 'n_estimators': [50, 100]},
             scoring='neg_root_mean_squared_error', verbose=3)

In [38]:
m2.best_estimator_

RandomForestRegressor(max_depth=10, random_state=12345)

In [39]:
m2.best_score_

-1953.3769419057007

Для случайного леса так же получили нужную нам оценку качества, хотя и чуть ниже чем у LightGBM. Оценка достагается при max_depth = 10.

Посмотрим сколько времени занимает обучение и предсказание лучших моделей.

In [40]:
%%time
best_model_lgb = m.best_estimator_
best_model_lgb.fit(features_train, target_train)

CPU times: user 5.44 s, sys: 32.7 ms, total: 5.47 s
Wall time: 5.44 s


LGBMRegressor(n_estimators=50, num_leaves=50, random_state=12345)

In [41]:
%%time
predictions_lgb = best_model_lgb.predict(features_test)

CPU times: user 559 ms, sys: 2.7 ms, total: 562 ms
Wall time: 598 ms


In [42]:
mean_squared_error(target_test, predictions_lgb)**0.5

1721.537990222616

In [43]:
%%time
best_model_forest = m2.best_estimator_
best_model_forest.fit(features_train_oe, target_train_oe)

CPU times: user 37.7 s, sys: 34.7 ms, total: 37.7 s
Wall time: 37.8 s


RandomForestRegressor(max_depth=10, random_state=12345)

In [44]:
%%time
predictions_forest = best_model_forest.predict(features_test_oe)

CPU times: user 760 ms, sys: 391 µs, total: 761 ms
Wall time: 767 ms


In [45]:
mean_squared_error(target_test_oe, predictions_forest)**0.5

1944.6699489501673

Провели обучение и посчитали RMSE на тестовых выборках для двух лучших моделей LightGBM и RandomForestRegressor. Обучение заняло меньше времени у LightGBM, а так же метрика качества оказалась лучше. Время предсказания примерно одинаковое.

Вывод.  
  
1. Была проведена предобработка данных. Удалены дубликаты, обработаны пропуски и аномалии, удалены лишние столбцы.  
2. Данные были подготовлены для обучения ими моделей. Для RandomForestRegressor был применен OrdinalEncoder(), а для LightGBM категориальные столбцы были приведены к правильному для него типу.  
3. С помощью GridSearchCV были найдены лучшие модели LightGBM и RandomForestRegressor. Ими оказались LGBMRegressor(n_estimators=50, num_leaves=50, random_state=12345) и RandomForestRegressor(max_depth=10, random_state=12345) соотвественно.  
4. Лучшие модели были заново обучены и проверилась метрика качества на тестовых данных. Получили, что обучение лучшей модели LightGBM заняло  5.47 s, а RandomForestRegressor 37.7 s. Предсказание для обоих моделей заняло меньше секунды, а результаты подсчета RMSE оказали следующие: для LightGBM 1721.537990222616, для RandomForestRegressor 1944.6699489501673.